# Manual model testing

This notebook contains a number of tests for the simulation model where the output must be interpreted by a user to determine if the model is functioning correctly.   This may include inspecting:

* a simulated log of events
* the distribution of an output
* a chart or plot produced by the model

## Imports

In [5]:
import numpy as np
import statistics

## Model imports

In [6]:
from s2_stroke_rehab_model import *

## Results collection tests

In [47]:
def test_results_collection_moments_1(audit_interval=1):
    '''
    Test ASU ward occupancy data collected is in a sensible range.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 0 to 50 with sensible moments.

    prints out:
    min
    max
    mean
    deciles

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    None
    '''

    # Create the experiment
    experiment = Experiment({
        'trace': False,
        'acute_audit_interval': audit_interval
    })

    # Create the simulation environment
    env = simpy.Environment()

    rehab_unit = RehabilitationUnit(env, experiment)


    # Create the AcuteStrokeUnit
    asu = AcuteStrokeUnit(env, experiment, rehab_unit)

    # Start the audit process - modified iteration 21
    env.process(experiment.audit_acute_occupancy(env, 1, audit_interval, asu, experiment))

    # Run the model - modified iteration 21
    asu.run()
    env.run(until=experiment.params['results_collection_period'])

    # Calculate statistics
    min_occupancy = min(experiment.asu_occupancy)
    max_occupancy = max(experiment.asu_occupancy)
    mean_occupancy = statistics.fmean(experiment.asu_occupancy)
    deciles = [round(q, 1) for q in statistics.quantiles(experiment.asu_occupancy, n=10)]

    # Print results
    print(f'Minimum occupancy: {min_occupancy}')
    print(f'Maximum occupancy: {max_occupancy}')
    print(f'Mean occupancy: {mean_occupancy}')
    print(f'Deciles of occupancy: {deciles}')


In [48]:
def test_results_collection_moments_2(audit_interval=1):
    '''
    Test REHAB ward occupancy data collected is in a sensible range.
    
    Expected result: The type collected is int. The values are in 
    the range 1 to [10-15] with sensible moments.

    Note this is when the Rehab unit is used independently
    with external arrivals only i.e. no transfers from ASU.

    prints out:
    min
    max
    mean
    deciles

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    None
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # create experiment
    experiment = Experiment()
    
    rehab_unit = RehabilitationUnit(env, experiment)
    rehab_unit.run()
    
    # Start the audit process - modified iteration 21
    env.process(experiment.audit_rehab_occupancy(env, 1, audit_interval, rehab_unit, experiment))
    
    # Run the model for the default run length in the experiment - modified in iteration 21
    env.run(until=experiment.params['results_collection_period'])
    
    print(f'min(experiment.rehab_occupancy)={min(experiment.rehab_occupancy)}')
    print(f'max(experiment.rehab_occupancy)={max(experiment.rehab_occupancy)}')
    print(f'statistics.fmean(experiment.rehab_occupancy)={statistics.fmean(experiment.rehab_occupancy)}')
    print('Deciles:', [round(q, 1) for q in statistics.quantiles(experiment.rehab_occupancy, n=10)])


In [49]:
def test_results_collection_moments_3(audit_interval=1):
    '''
    SYSTEM TEST
    
    Test REHAB ward occupancy data collected is in a sensible range
    when it is connected to the ASU model.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 1 to [10-15] with sensible moments.

    Expected result: 
        len(experiment.asu_occupancy) == env.now - 1 AND
        len(experiment.rehab_occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Create an experiment with default parameters
    experiment = Experiment()

    # Create models
    rehab_unit = RehabilitationUnit(env, experiment)
    asu = AcuteStrokeUnit(env, experiment, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    asu.run()
    rehab_unit.run()

    # Start the audit process - modified iteration 21
    env.process(experiment.audit_rehab_occupancy(env, 1, audit_interval, rehab_unit, experiment))
    env.process(experiment.audit_acute_occupancy(env, 1, audit_interval, asu, experiment))
    
    # Run the simulation until the specified run length in the Experiment parameters - modified in iteration 21
    env.run(until=experiment.params['results_collection_period']*10)

    # Print info for debug
    print(f'min(experiment.rehab_occupancy)={min(experiment.rehab_occupancy)}')
    print(f'max(experiment.rehab_occupancy)={max(experiment.rehab_occupancy)}')
    print(f'statistics.fmean(experiment.rehab_occupancy)={statistics.fmean(experiment.rehab_occupancy)}')
    print('Deciles:', [round(q, 1) for q in statistics.quantiles(experiment.rehab_occupancy, n=10)])

    # Test
    return len(experiment.asu_occupancy) == (env.now - 1) and \
           len(experiment.rehab_occupancy) == (env.now - 1)


## Test suppress trace

These tests check that the simulated log can be suppressed when the model is running

In [53]:
def test_suppress_log_asu(trace):
    '''
    Test that setting experiment.trace 
    False suppresses output.

    Expected result: patient_count > 0

    Params:
    ------
    trace: bool
        is trace outputted or not?

    Returns:
    -------
    int: number of patient arrivals.
    '''
    experiment = Experiment({'trace': trace})
    env = simpy.Environment()
    rehab_unit = RehabilitationUnit(env, experiment)
    model = AcuteStrokeUnit(env, experiment, rehab_unit)
    model.run()
    env.run(until=experiment.params['results_collection_period'])
    
    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

In [57]:
def test_suppress_log_rehab(trace):
    '''
    Test that setting experiment.trace to
    False suppresses output in the rehab model

    Expected result: patient_count > 0

    Params:
    ------
    trace: bool
        is trace outputted or not?

    Returns:
    -------
    int: number of patient arrivals.
    '''
    experiment = Experiment({'trace': trace})
    env = simpy.Environment()
    model = RehabilitationUnit(env, experiment)
    asu = AcuteStrokeUnit(env, experiment, model)
    model.run()
    env.run(until=100)
    
    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    print(f"Final rehab patient counts: {model.patient_counts}")

### Extreme value tests

Extreme value tests are used pragmatically to block of routes/arrivals/activites in the simulation model and check the results.

The most simple way to modify the model for these tests is to set parameters to $M$ a very large number.

In [59]:
def ev_test_1(large_number):
    '''
    TIA, Complex Neuro, Other, have their inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model 
    is "Stroke". This is seen in the event log

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    # Create the simulation environment
    def main(arrival_rates=None, results_collection_period=None):
        custom_params = {}
        
        if arrival_rates:
            custom_params['patient_types'] = arrival_rates
        
        if results_collection_period:
            custom_params['results_collection_period'] = results_collection_period
    
        # Create an Experiment instance with custom parameters
        experiment = Experiment(custom_params)
    
        # Run the simulation with the custom experiment
        env = simpy.Environment()
        rehab_unit = RehabilitationUnit(env, experiment)
        model = AcuteStrokeUnit(env, experiment, rehab_unit)
        model.run()
        env.run(until=experiment.params['results_collection_period'])
    
        # Print results to check
        print("\nSimulation completed.")
        print(f"Total simulation time: {model.env.now:.2f} days")
        print(f"Total patient arrivals: {model.total_arrivals}")
        for patient_type in model.patient_types.values():
            print(f"Total {patient_type.name} arrivals: {patient_type.count}")
    
    if __name__ == "__main__":
        large_number = M
        
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': 1.2},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
    
    main(asu_arrival_rates, results_collection_period=2*365)  # Run for 2 years with custom parameters


## Run tests

### Results collection moments 1 (ASU only)

In [50]:
test_results_collection_moments_1()

Minimum occupancy: 1
Maximum occupancy: 21
Mean occupancy: 8.296600877192983
Deciles of occupancy: [5.0, 6.0, 7.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]


### Results collection moments 2 (REHAB only)

In [51]:
test_results_collection_moments_2()

min(experiment.rehab_occupancy)=0
max(experiment.rehab_occupancy)=8
statistics.fmean(experiment.rehab_occupancy)=2.633771929824561
Deciles: [1.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 4.0, 5.0]


### Results collection moment 3 (ASU+REHAB)

In [52]:
test_results_collection_moments_3()

min(experiment.rehab_occupancy)=0
max(experiment.rehab_occupancy)=21
statistics.fmean(experiment.rehab_occupancy)=9.399145158638829
Deciles: [6.0, 7.0, 8.0, 9.0, 9.0, 10.0, 11.0, 12.0, 13.0]


True

### Test suppress trace
The following tests should only output statistics after the model run. There should be no printed log from within the run.

In [54]:
test_suppress_log_asu(trace=False)


Simulation completed.
Total simulation time: 1825.00 days
Total patient arrivals: 2821
Total Stroke arrivals: 1525
Total TIA arrivals: 216
Total Complex Neurological arrivals: 541
Total Other arrivals: 539


In [58]:
test_suppress_log_rehab(trace=False)


Simulation completed.
Total simulation time: 100.00 days
Total patient arrivals: 8
Final rehab patient counts: {'Stroke': 4, 'Complex Neurological': 1, 'Other': 3, 'TIA': 0}


### Extreme value tests

In [60]:
M = 10_000_000

In [61]:
ev_test_1(M)


Simulation completed.
Total simulation time: 730.00 days
Total patient arrivals: 607
Total Stroke arrivals: 607
Total TIA arrivals: 0
Total Complex Neurological arrivals: 0
Total Other arrivals: 0
